In [20]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

ModuleNotFoundError: No module named 'google'

In [ ]:
%cd /content/drive/MyDrive/OptiML

In [ ]:
! pip install foolbox==3.3.1

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import sklearn
import torch
import torchvision
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torch.utils.data.sampler import SubsetRandomSampler
from torchvision import datasets, transforms
from sklearn.metrics import accuracy_score, recall_score, f1_score
from Functions.implementations import *
from Functions.visualization import *
import foolbox as fb
from tqdm import tqdm
import os

In [2]:
def attack_n_batches(model, test_loader, device, n_batches=3, attack = fb.attacks.BoundaryAttack()):
    """
    Run a black-box attack on a model for a specified number of batches.
    
    Args:
        model: The model to attack.
        test_loader: DataLoader for the test dataset.
        n_batches: Number of batches to process.
        
    Returns:
        success_rate: Success rate of the attack.
        avg_perturbation: Average perturbation of the adversarial examples.
        avg_queries: Average number of queries made (if applicable).
    """
    # Ensure the model is in evaluation mode
    fmodel = fb.PyTorchModel(model, bounds=(0, 1))
    
    # Initialize lists to store results
    successes, perturbations, queries = [], [], []

    # Iterate over three batches of images
    for i in tqdm(range(n_batches), desc="Attacking model"):
        images, labels = next(iter(test_loader))
        images, labels = images.to(device), labels.to(device)
        # Optionally, filter out misclassified samples
        preds = model(images).argmax(dim=1)
        mask = preds == labels
        if mask.sum() == 0:
            continue
        images, labels = images[mask], labels[mask]
        # Run attack
        raw_advs, clipped_advs, success = attack(fmodel, images, labels, epsilons=None)
        # Collect metrics
        successes.append(success.cpu())
        perturbations.append((clipped_advs - images).view(images.size(0), -1).norm(dim=1).cpu())
        # If available, track queries (depends on attack implementation)
        queries.append(attack.queries.cpu()) if hasattr(attack, 'queries') else None
    # Aggregate and report
    if successes:
        success_rate = torch.cat(successes).float().mean().item()
        avg_perturbation = torch.cat(perturbations).mean().item()
        avg_queries = torch.cat(queries).float().mean().item() if queries else None
    else:
        print("No successful batches processed. Check your data or model accuracy.")
        success_rate, avg_perturbation, avg_queries = None, None, None
    return success_rate, avg_perturbation, avg_queries

# extract the learning rate, beta_1, and beta_2 from the model name
def extract_params(model_name):
    parts = model_name.split('_')
    learning_rate = float(parts[2])
    beta_1 = float(parts[4])
    beta_2 = float(parts[6][:-4])
    return learning_rate, beta_1, beta_2

In [3]:
set_seed(42)

n_batches = 16
batch_size = 16
_, _, test_loader = get_data_loaders(batch_size=batch_size)

drive_base_path = os.getcwd()
folder_path = os.path.join(drive_base_path, 'Results/Adam_densenet')
csv_path = os.path.join(folder_path, 'densenetBlackBox_topmodel.csv')

#model_list = ['VGG_lr_0.0005_beta1_0.9_beta2_0.98.pth']#, 'VGG_lr_0.0005_beta1_0.8_beta2_0.99.pth', 'VGG_lr_0.0005_beta1_0.8_beta2_0.98.pth'] # for VGG models
#model_list = ['resnet_lr_0.001_beta1_0.8_beta2_0.999.pth']#, 'resnet_lr_0.001_beta1_0.9_beta2_0.99.pth', 'resnet_lr_0.001_beta1_0.8_beta2_0.9999.pth'] # for resnet models
model_list = ['densenet_lr_0.001_beta1_0.8_beta2_0.9999.pth']#, 'densenet_lr_0.0005_beta1_0.9_beta2_0.999.pth', 'densenet_lr_0.001_beta1_0.9_beta2_0.9999.pth'] # for densenet models

attack_list = [
    fb.attacks.BoundaryAttack(),
    fb.attacks.GaussianBlurAttack(),
    fb.attacks.SaltAndPepperNoiseAttack()
]

if not os.path.exists(csv_path):
    columns = ["file_name", "learning_rate", "beta_1", "beta_2", "attack_type","success_rate","avg_perturbations","avg_queries"]
    pd.DataFrame(columns=columns).to_csv(csv_path, index=False)

for model_name in model_list:
    if os.path.exists(os.path.join(folder_path, model_name)):
        print(f"File {model_name} exists in the folder.")
    else:
        print(f"File {model_name} does not exist in the folder. Please check the path or file name.")
        continue
    print("Loading model: ", model_name)
    if not model_name.endswith('.pth'):
        continue
    model_path = os.path.join(folder_path, model_name)
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    #model = VGGLike().to(device) # for VGG models
    #model = get_resnet18_cifar().to(device) # for resnet models
    model = get_densenet121().to(device) # for densenet models
    model.load_state_dict(torch.load(model_path, map_location=device))
    model.eval()
    for attack in attack_list:
        print(f"Running attack: {attack.__class__.__name__}")
        results = attack_n_batches(model, test_loader, device, n_batches, attack)
        print("Saving results")
        df = pd.DataFrame([model_name] + list(extract_params(model_name)) + [attack.__class__.__name__] + list(results)).T
        df.to_csv(csv_path, mode='a', header=False, index=False)
        clear_output(wait=True)
    

File densenet_lr_0.001_beta1_0.8_beta2_0.9999.pth exists in the folder.
Loading model:  densenet_lr_0.001_beta1_0.8_beta2_0.9999.pth
Running attack: BoundaryAttack


Attacking model:   0%|          | 0/16 [1:56:24<?, ?it/s]


KeyboardInterrupt: 